In [1]:
import json
import copy

In [2]:

fs = open('gameData.json', 'r')
gameData = json.load(fs)
fs.close()


weapons = gameData['weapons']

In [3]:
weapons_traits = [weapon['trait'].split(", ") for weapon in weapons]
new_weapons_traits = []

for i, weapon_traits in enumerate(weapons_traits):
  new_Weapon_traits = []
  for trait in weapon_traits:
    trait = trait.replace('형', '')
    new_Weapon_traits.append(trait)

  new_weapons_traits.append(new_Weapon_traits)


new_weapons = []

for i, weapon in enumerate(weapons):
  new_weapon = copy.deepcopy(weapon)
  new_weapon['trait'] = new_weapons_traits[i]
  new_weapons.append(new_weapon)
  
weapons = new_weapons

In [4]:
weapons_damages = [weapon['damage'].split('\n') for weapon in weapons]
weapons_damages = [[weapon_damage.strip() for weapon_damage in weapon_damages if weapon_damage.strip() != ''] for weapon_damages in weapons_damages]
# ['1d6+1', '역장'], 한개 모양 다름
new_weapons_damages = [[weapon_damage.replace('+ ', '+') for weapon_damage in weapon_damages] for weapon_damages in weapons_damages]
new_weapons = []

for i, weapon in enumerate(weapons):
  new_weapon = copy.deepcopy(weapon)
  new_weapon['damage'] = new_weapons_damages[i]
  new_weapons.append(new_weapon)
  
weapons = new_weapons

In [5]:
newWeapons = []

for weapon in weapons:
  newWeapon = copy.deepcopy(weapon)
  newRange = float(newWeapon['weaponRange'].replace('m', ''))
  newWeapon['weaponRange'] = newRange
  newWeapons.append(newWeapon)

weapons = newWeapons

In [6]:
newWeapons = []

for weapon in weapons:
  newWeapon = copy.deepcopy(weapon)
  newEnchantment = int(newWeapon['enchantment'])
  newWeapon['enchantment'] = newEnchantment
  newWeapons.append(newWeapon)

weapons = newWeapons

In [7]:
def getDamage(numberOfDice: int, sidesOfDice: int, constant_damage = 0):
  max_damage = numberOfDice * sidesOfDice + constant_damage
  min_damage = numberOfDice + constant_damage
  mean_damage = (1 + sidesOfDice) / 2 * numberOfDice
  
  return {'maxDamage': max_damage, 'minDamage': min_damage, 'meanDamage': mean_damage}

In [8]:
def getMainDamage(mainDamage: str):
  constant_damage = 0
  mainDamage = mainDamage.split('+')
  
  if(len(mainDamage) == 2): constant_damage += int(mainDamage[1])
  mainDamage = mainDamage[0]

  if('(' in mainDamage): mainDamage = mainDamage.split('(')[0]

  splitedDice = mainDamage.split('d')
  numberOfDice = int(splitedDice[0])
  sidesOfDice = int(splitedDice[1])

  return getDamage(numberOfDice, sidesOfDice, constant_damage)

In [9]:
def getAdditionalDamage(addedDamage: str):
  newAddedDamage = addedDamage.replace('+', '')
  newAddedDamage = newAddedDamage.split('d')

  numberOfDice = int(newAddedDamage[0])
  sidesOfDice = int(newAddedDamage[1])

  return getDamage(numberOfDice, sidesOfDice)

In [10]:
def getWeaponDamage(weapon):
  maxDamage = 0
  minDamage = 0
  meanDamage = 0

  damages = weapon['damage']
  damages = ' '.join(damages)
  damages = damages.split(' ')
  damages = [damage for damage in damages if('d' in damage)]

  default_damage = [damage for damage in damages if(damage[0] != '+')]
  default_damage = default_damage[0]

  added_damages = [damage for damage in damages if(damage[0] == '+')]

  mainDamage = getMainDamage(default_damage)
  aditionalDamage = [getAdditionalDamage(damage) for damage in added_damages]

  maxDamage += mainDamage['maxDamage']
  minDamage += mainDamage['minDamage']
  meanDamage += mainDamage['meanDamage']

  for damage in aditionalDamage:
    maxDamage += damage['maxDamage']
    minDamage += damage['minDamage']
    meanDamage += damage['meanDamage']

  return {'maxDamage': maxDamage, 'minDamage': minDamage, 'meanDamage': meanDamage};


In [11]:
newWeapons = []

for i, weapon in enumerate(weapons):
  newWeapon = copy.deepcopy(weapon)
  newWeapon['damageStat'] = weapon_damage = getWeaponDamage(weapon)
  newWeapons.append(newWeapon)

weapons = newWeapons

In [12]:
newWeapons = []

for weapon in weapons:
  newWeapon = copy.deepcopy(weapon)
  newWeapon['info'] = ' '.join([ token for token in newWeapon['info'].split(' ') if(token != '')]).replace('\n', '')
  newWeapons.append(newWeapon)

weapons = newWeapons



In [13]:
newWeapons = []

for weapon in weapons:
  newWeapon = copy.deepcopy(weapon)
  newWeapon['special'] = ' '.join([ token.strip() for token in newWeapon['special'].split(' ') if(token != '')]).split('  ')
  if(newWeapon['special'][0] == '-'): newWeapon['special'] = []
  newWeapons.append(newWeapon)

weapons = newWeapons




In [14]:
newWeapons = []

for weapon in weapons:
  newWeapon = copy.deepcopy(weapon)
  newWeapon['weaponActions'] = ' '.join([token.lstrip() for token in newWeapon['weaponActions'].split(' ') if(token != '')])
  if(':' in newWeapon['weaponActions'].split('\n')[0]):
    defaultWeaponActions = []
    addtionalWeaponActions = newWeapon['weaponActions'].split('\n')[0:]
  else :
    defaultWeaponActions = newWeapon['weaponActions'].split('\n')[0].replace(',',' ').split('  ')
    addtionalWeaponActions = newWeapon['weaponActions'].split('\n')[1:]
  
  addtionalWeaponActions = ' '.join(addtionalWeaponActions)
  addtionalWeaponActions = addtionalWeaponActions.strip().replace('  ', ' ').split('  ')

  weaponActions = defaultWeaponActions + addtionalWeaponActions
  weaponActions = [weaponAction for weaponAction in weaponActions if(weaponAction != '' and weaponAction != '-')]

  newWeapon['weaponActions'] = weaponActions
  newWeapons.append(newWeapon)

weapons = newWeapons

In [15]:
newWeapons = []

for weapon in weapons:
  newWeapon = copy.deepcopy(weapon)
  if(newWeapon['info'] == ''): newWeapon['info'] = '-'
  newWeapons.append(newWeapon)

weapons = newWeapons
  


In [21]:
for i, weapon in enumerate(weapons):
  weapon['id'] = i + 1

In [23]:
# fs = open('gameData_v03.json', 'w')
# json.dump(weapons, fs, ensure_ascii=False)
# fs.close()